In [2]:
import os
import cv2
import pydicom
import numpy as np
import pandas as pd
from pydicom.pixel_data_handlers.util import apply_voi_lut

# Rutas al dataset y archivos
dicom_root = '/Volumes/m2/Memoria/Code/PMM/VinDr-Mammo-Preprocessing/data/processed/subset_datos/Images_Mass_Calc'
csv_path = '/Volumes/m2/Memoria/Code/PMM/VinDr-Mammo-Preprocessing/data/processed/csv/filtered_with_image_names.csv'
output_root = '/Volumes/m2/Memoria/Code/PMM/VinDr-Mammo-Preprocessing/data/processed/roi_images/ROICROP4'

# Crear carpetas principales (training y test)
train_folder = os.path.join(output_root, 'training')
test_folder = os.path.join(output_root, 'test')

os.makedirs(train_folder, exist_ok=True)
os.makedirs(test_folder, exist_ok=True)

# Crear carpetas de masas y calcificaciones, y sus subcarpetas (benigno, maligno, sospechoso) dentro de 'training' y 'test'
for split_folder in [train_folder, test_folder]:
    os.makedirs(os.path.join(split_folder, 'masas', 'benigno'), exist_ok=True)
    os.makedirs(os.path.join(split_folder, 'masas', 'maligno'), exist_ok=True)
    os.makedirs(os.path.join(split_folder, 'masas', 'sospechoso'), exist_ok=True)
    os.makedirs(os.path.join(split_folder, 'calcificaciones', 'benigno'), exist_ok=True)
    os.makedirs(os.path.join(split_folder, 'calcificaciones', 'maligno'), exist_ok=True)
    os.makedirs(os.path.join(split_folder, 'calcificaciones', 'sospechoso'), exist_ok=True)

# Tamaño objetivo al que queremos llegar para el recorte (299x299 píxeles)
TARGET_SIZE = (299, 299)

# Leer las anotaciones CSV que contienen las coordenadas de la ROI
ss1 = pd.read_csv(csv_path)

# Limpiar las categorías eliminando los caracteres adicionales
ss1['finding_categories'] = ss1['finding_categories'].apply(lambda x: x.strip("[]'"))

# Crear un DataFrame vacío para guardar las nuevas coordenadas
ss1_rescaled = pd.DataFrame(columns=ss1.columns)

# Función para obtener la ruta del archivo DICOM usando image_id
def get_dicom_path(image_id):
    for study_id in os.listdir(dicom_root):
        study_path = os.path.join(dicom_root, study_id)
        if os.path.isdir(study_path):
            dicom_path = os.path.join(study_path, image_id + '.dicom')
            if os.path.exists(dicom_path):
                return dicom_path
    raise FileNotFoundError(f"No se encontró el archivo DICOM para image_id: {image_id}")

# Función para aplicar el recorte basado en el centro de la ROI, con padding variable.
def extract_roi(image_name, split, birads, finding_categories, ss1_rescaled):
    # Obtener el image_id (parte antes del guión bajo "_")
    image_id = image_name.split('_')[0]
    dicom_path = get_dicom_path(image_id)

    # Leer la imagen DICOM
    dicom = pydicom.dcmread(dicom_path)
    original_image = dicom.pixel_array

    # Aplicar VOI LUT con prefer_lut=True (priorizando LUT si está presente)
    img_windowed = apply_voi_lut(original_image, dicom, prefer_lut=True)

    # Normalizar la imagen para que esté en el rango [0, 255]
    img_windowed = (img_windowed - img_windowed.min()) / (img_windowed.max() - img_windowed.min()) * 255
    img_windowed = img_windowed.astype(np.uint8)

    # Extraer las coordenadas de la ROI desde el CSV utilizando el image_name
    try:
        x1, y1 = (int(ss1.loc[ss1['image_name'] == image_name, 'xmin'].values[0]),
                  int(ss1.loc[ss1['image_name'] == image_name, 'ymin'].values[0]))
        x2, y2 = (int(ss1.loc[ss1['image_name'] == image_name, 'xmax'].values[0]),
                  int(ss1.loc[ss1['image_name'] == image_name, 'ymax'].values[0]))
    except IndexError:
        print(f"Error: No se encontraron coordenadas para la ROI de la imagen {image_name}.")
        return ss1_rescaled

    # Verificar que las coordenadas sean válidas
    x1 = max(0, x1)
    y1 = max(0, y1)
    x2 = min(img_windowed.shape[1], x2)
    y2 = min(img_windowed.shape[0], y2)
    
    # Calcular el centro de la ROI
    center_x = (x1 + x2) // 2
    center_y = (y1 + y2) // 2

    # Determinar el tamaño de la ROI original
    width = x2 - x1
    height = y2 - y1

    # Agregar padding variable dependiendo de cuál eje es mayor y el tamaño de la ROI
    if width > height:
        # ROI más ancha, aplicar padding solo en el eje Y
        if width < TARGET_SIZE[0]:
            # Si es menor a 299, aplicar un 20% de padding en el eje Y
            padding_y = int(height * 0.2)
        else:
            # Si es mayor o igual a 299, aplicar un 10% de padding en el eje Y
            padding_y = int(height * 0.1)
        padding_x = 0  # No se aplica padding en el eje X
    else:
        # ROI más alta, aplicar padding solo en el eje X
        if height < TARGET_SIZE[1]:
            # Si es menor a 299, aplicar un 20% de padding en el eje X
            padding_x = int(width * 0.2)
        else:
            # Si es mayor o igual a 299, aplicar un 10% de padding en el eje X
            padding_x = int(width * 0.1)
        padding_y = 0  # No se aplica padding en el eje Y

    # Establecer los nuevos límites de la imagen recortada
    x1_new = max(center_x - (width // 2) - padding_x, 0)
    y1_new = max(center_y - (height // 2) - padding_y, 0)
    x2_new = min(center_x + (width // 2) + padding_x, img_windowed.shape[1])
    y2_new = min(center_y + (height // 2) + padding_y, img_windowed.shape[0])

    # Recortar la imagen utilizando las nuevas coordenadas ajustadas
    crop = img_windowed[y1_new:y2_new, x1_new:x2_new]

    # Verificar si el recorte no está vacío
    if crop.size == 0:
        print(f"Error: El recorte para la imagen {image_name} está vacío.")
        return ss1_rescaled

    # Redimensionar el recorte a 299x299 píxeles
    try:
        crop_resized = cv2.resize(crop, TARGET_SIZE)
    except cv2.error as e:
        print(f"Error al redimensionar el recorte para la imagen {image_name}: {e}")
        return ss1_rescaled

    # Calcular las nuevas coordenadas reescaladas
    scale_x = TARGET_SIZE[0] / crop.shape[1]
    scale_y = TARGET_SIZE[1] / crop.shape[0]

    xmin_rescaled = int((x1 - x1_new) * scale_x)
    ymin_rescaled = int((y1 - y1_new) * scale_y)
    xmax_rescaled = int((x2 - x1_new) * scale_x)
    ymax_rescaled = int((y2 - y1_new) * scale_y)

    # Ajuste en el split: manejar 'train', 'training' y 'test'
    if split in ['train', 'training']:
        base_dir = train_folder
    elif split == 'test':
        base_dir = test_folder
    else:
        print(f"Error: Split desconocido '{split}' para la imagen {image_name}.")
        return ss1_rescaled

    # Verificar si se asigna una categoría válida a la variable output_dir
    if finding_categories == 'Mass':
        output_dir = os.path.join(base_dir, 'masas')
    elif finding_categories == 'Suspicious Calcification':
        output_dir = os.path.join(base_dir, 'calcificaciones')
    else:
        print(f"Error: Categoría desconocida '{finding_categories}' para la imagen {image_name}.")
        return ss1_rescaled

    # Categorizar en benigno, maligno, o sospechoso según el valor de BIRADS
    if birads == 'BI-RADS 3':
        output_dir = os.path.join(output_dir, 'benigno')
    elif birads == 'BI-RADS 5':
        output_dir = os.path.join(output_dir, 'maligno')
    elif birads == 'BI-RADS 4':
        output_dir = os.path.join(output_dir, 'sospechoso')
    else:
        print(f"Error: BIRADS desconocido '{birads}' para la imagen {image_name}.")
        return ss1_rescaled

    # Guardar la imagen recortada con el formato {image_name}.png
    roi_filename = f"{image_name}.png"
    output_path = os.path.join(output_dir, roi_filename)
    cv2.imwrite(output_path, crop_resized)
    print(f"Imagen ROI guardada en: {output_path}")

    # Guardar las coordenadas reescaladas en el nuevo CSV
    new_row = ss1.loc[ss1['image_name'] == image_name].copy()
    new_row['xmin'] = xmin_rescaled
    new_row['ymin'] = ymin_rescaled
    new_row['xmax'] = xmax_rescaled
    new_row['ymax'] = ymax_rescaled
    ss1_rescaled = pd.concat([ss1_rescaled, new_row], ignore_index=True)

    return ss1_rescaled

# Función para recorrer todas las imágenes del CSV y guardar las ROI redimensionadas y las nuevas coordenadas
def process_all_images():
    global ss1_rescaled
    for _, row in ss1.iterrows():
        image_name = row['image_name']
        split = row['split']  # La columna 'split' tiene los valores 'train', 'training', o 'test'
        birads = row['finding_birads']  # Columna con los valores BIRADS
        finding_categories = row['finding_categories']  # Columna con el tipo de hallazgo (mass o suspicious)

        # Procesar cada imagen y almacenar las ROI y coordenadas reescaladas
        ss1_rescaled = extract_roi(image_name, split, birads, finding_categories, ss1_rescaled)

    # Guardar el nuevo CSV con las coordenadas reescaladas
    new_csv_path = '/Volumes/m2/Memoria/Code/PMM/VinDr-Mammo-Preprocessing/data/processed/csv/CROP4.csv'
    ss1_rescaled.to_csv(new_csv_path, index=False)
    print(f"Nuevo CSV guardado en: {new_csv_path}")

# Ejecutar el procesamiento de todas las imágenes
process_all_images()


Imagen ROI guardada en: /Volumes/m2/Memoria/Code/PMM/VinDr-Mammo-Preprocessing/data/processed/roi_images/ROICROP4/training/masas/sospechoso/001ade2a3cb53fd808bd2856a0df5413_0.png
Imagen ROI guardada en: /Volumes/m2/Memoria/Code/PMM/VinDr-Mammo-Preprocessing/data/processed/roi_images/ROICROP4/training/calcificaciones/sospechoso/001ade2a3cb53fd808bd2856a0df5413_1.png
Imagen ROI guardada en: /Volumes/m2/Memoria/Code/PMM/VinDr-Mammo-Preprocessing/data/processed/roi_images/ROICROP4/training/calcificaciones/sospechoso/001ade2a3cb53fd808bd2856a0df5413_2.png
Imagen ROI guardada en: /Volumes/m2/Memoria/Code/PMM/VinDr-Mammo-Preprocessing/data/processed/roi_images/ROICROP4/training/calcificaciones/sospechoso/002074412a8fc178c271fb93b55c3e29_0.png
Imagen ROI guardada en: /Volumes/m2/Memoria/Code/PMM/VinDr-Mammo-Preprocessing/data/processed/roi_images/ROICROP4/training/masas/benigno/002460132586dc0c7b88a59dce6e77bd_0.png
Imagen ROI guardada en: /Volumes/m2/Memoria/Code/PMM/VinDr-Mammo-Preprocessing

/var/folders/ds/4z7lc9ld4mqb377_3t2v4lxm0000gn/T/ipykernel_60409/4287808862.py:183: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  ss1_rescaled = pd.concat([ss1_rescaled, new_row], ignore_index=True)


Imagen ROI guardada en: /Volumes/m2/Memoria/Code/PMM/VinDr-Mammo-Preprocessing/data/processed/roi_images/ROICROP4/training/calcificaciones/sospechoso/0171ab32059f4c226164a13c311f6824_0.png
Imagen ROI guardada en: /Volumes/m2/Memoria/Code/PMM/VinDr-Mammo-Preprocessing/data/processed/roi_images/ROICROP4/test/masas/sospechoso/01958718afdf303581e758cdf34eaf8a_0.png
Imagen ROI guardada en: /Volumes/m2/Memoria/Code/PMM/VinDr-Mammo-Preprocessing/data/processed/roi_images/ROICROP4/training/masas/maligno/01df962b078e38500bf9dd9969a50083_0.png
Imagen ROI guardada en: /Volumes/m2/Memoria/Code/PMM/VinDr-Mammo-Preprocessing/data/processed/roi_images/ROICROP4/training/masas/benigno/01fb871dc222684a9950609b62b76772_0.png
Imagen ROI guardada en: /Volumes/m2/Memoria/Code/PMM/VinDr-Mammo-Preprocessing/data/processed/roi_images/ROICROP4/training/masas/benigno/02895cbbcb504f54c6882c4c5695f563_0.png
Imagen ROI guardada en: /Volumes/m2/Memoria/Code/PMM/VinDr-Mammo-Preprocessing/data/processed/roi_images/ROI

Detección del eje mayor: Se comprueba si la ROI es más grande en el eje X o en el eje Y.


Si la ROI es más grande en el eje X, se aplica padding en el eje Y para mantener las proporciones.
Si la ROI es más grande en el eje Y, se aplica padding en el eje X.

Padding dinámico:

20% de padding si el tamaño del eje correspondiente es menor a 299.
10% de padding si el tamaño del eje correspondiente es mayor o igual a 299.

Almacenamiento:


Este ajuste permitirá que el padding se aplique de manera proporcional al tamaño de la ROI, manteniendo la lógica de aplicar padding solo en un eje y sin deformar la imagen.
